In [ ]:
import numpy as np
import scipy.signal as sig
import bokeh.plotting as bkp
import bokeh.models as bkm
bkp.output_notebook()

import os
os.environ['ARRAY_MODULE'] = 'numpy'
from asl_bloch_sim import bloch, rf, flow
from asl_bloch_sim import xp, asnumpy

In [ ]:
np.linspace(0, 0.25, 17)

In [ ]:
np.linspace(0, 1.5, 21)

In [ ]:
dt = 0.00002 # seconds
# PCASL: A typical real sequence may use 750 0.5 ms, 20º, Hann RF pulses over a 1500 ms period
# 750 pulses with these durations corresponds to a duty cycle of 0.5
duration = 2.2 # 2.5 # seconds
label_duration = 2 # seconds
num_reps = 2500 # 1333

flip_angle = 20 # degrees
rf_duration = 0.0005 # seconds
rf_bandwidth = 500 # Hz

G_max = 15e-3 # T/m
G_avg = 1e-3 # T/m
S_max = 150 # T/m/s # look up max skew rate for your scanner

off_resonance = 2000 # Hz
spectrum_lines = 1 # 5
B1_homogeneity = np.array([1]) # np.linspace(0.5, 1, 5) # fraction of B1

num_flow = 50000
position_offset = 0.1 # 0.04 # m
diastolic_velocity = np.linspace(0, 0.25, 26) # 0.137 # m/s
systolic = np.linspace(0, 1.5, 31) # m/s
radial, vessel_cross_section = flow.speed_across_vessel(normalized=True, num=11)
flow_time, flow_velocity, flow_position = flow.holdsworth_cca(stop=duration, # + 0.5,
                                                              num=round(duration / dt),
                                                              cross_section=vessel_cross_section,
                                                              systolic_velocity=systolic,
                                                              diastolic_velocity=diastolic_velocity)
flow_position -= position_offset

T1 = 1.65 # seconds # https://doi.org/10.1002/mrm.25197
T2 = 0.186 # seconds # https://doi.org/10.1002/mrm.21858

In [ ]:
G_max / G_avg

In [ ]:
flow_position.shape

In [ ]:
# bokeh plot for trajectory
p = bkp.figure(width=800, height=300, title='Blood flow', x_range=(flow_time[0], flow_time[-1]))
for i in range(flow_position.shape[2]):
       p.line(flow_time, flow_position[:, 3, i, 0], alpha=1 - i/flow_position.shape[2], line_width=2, legend_label='Position')
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Position (m)'
p.line([flow_time[0], flow_time[-1]], [0] * 2, line_color='red', line_width=2,
       line_dash='dashed', legend_label='Labeling plane')
p.legend.click_policy = 'hide'
p.legend.location = 'top_left'
# bkp.output_file('blood_trajectory.html')
# bkp.save(p)
bkp.show(p)


In [ ]:
labelling_plane_thickness = rf_bandwidth / (bloch.GAMMA_BAR * G_max) # m
labelling_plane_thickness * 1e3 # mm

In [ ]:
DeltaT = label_duration / num_reps
DeltaT * 1e6 # µs

In [ ]:
1/DeltaT # Hz

In [ ]:
rf_duration / DeltaT

In [ ]:
G_min = (G_avg - G_max * (rf_duration / DeltaT)) / (1 - rf_duration/DeltaT)
G_min # T/m

In [ ]:
# bokeh plot for blood flow
p = bkp.figure(width=800, height=300, title='Blood flow')
p.line(flow_time, flow_velocity[:, 0, -1, 0], line_width=2)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Speed (m/s)'
bkp.show(p)

In [ ]:
# bokeh plot for trajectory
p = bkp.figure(width=800, height=300, title='Blood flow', x_range=(flow_time[0], flow_time[-1]))
p.line(flow_time, flow_position[:, 0, -1, 0], line_width=2, legend_label='Position')
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Position (m)'
p.line([flow_time[0], flow_time[-1]], [0] * 2, line_color='red', line_width=2,
       line_dash='dashed', legend_label='Labeling plane')
p.legend.click_policy = 'hide'
p.legend.location = 'top_left'
# bkp.output_file('blood_trajectory.html')
# bkp.save(p)
bkp.show(p)


In [ ]:
T2 / dt # >> 1

In [ ]:
(G_max / G_avg) / (DeltaT / rf_duration) # >> 1 for no aliased labelling planes

In [ ]:
time = np.arange(0, duration, dt) # seconds
rf_time = np.arange(-rf_duration / 2, rf_duration / 2, dt)

rf_pulse = rf.sinc_pulse(flip_angle, rf_duration, rf_bandwidth, dt, phase_angle=90)
rf_period = rf.extend(rf_pulse, label_duration / num_reps, dt)
rf_label = np.tile(rf_period, num_reps)
rf_sig = rf.extend(rf_label, duration, dt)

G_period = np.append(np.full_like(rf_time, G_max), np.full(round((DeltaT - rf_duration) / dt), G_min))
G = rf.extend(np.tile(G_period, num_reps), duration, dt)[:, np.newaxis, np.newaxis, np.newaxis, np.newaxis]

dfz = np.linspace(0, off_resonance, spectrum_lines) # Hz
B = bloch.construct_B_field(rf_sig, G, flow_position[..., np.newaxis], off_resonance=dfz, B1_sensitivity=B1_homogeneity)

In [ ]:
(1 / (bloch.GAMMA_BAR * np.abs(rf_sig).max())) / dt # >> 1

In [ ]:
np.abs(rf_sig).max()

In [ ]:
np.abs(rf_pulse).mean()

In [ ]:
abs(rf_sig.mean()) * 1e6 # µT

In [ ]:
velo_at_plane = np.take_along_axis(flow_velocity, np.abs(flow_position).argmin(axis=0)[np.newaxis], axis=0)[0]
velo_at_plane[0, -1, 0] # m/s


In [ ]:
time_at_plane = time[np.abs(flow_position).argmin(axis=0)]
time_at_plane[0, -1, 0] # s

In [ ]:
B.shape

In [ ]:
B.size * 4 / 1e9 # GB

In [ ]:
type(B)

In [ ]:
# plot RF with bokeh
plot = bkp.figure(width=800, height=400, title='RF pulse')
plot.line(rf_time * 1e3, rf_pulse.real * 1e6, line_width=2)
plot.line(rf_time * 1e3, rf_pulse.imag * 1e6, line_width=2, color='orange')
plot.xaxis.axis_label = 'Time (ms)'
plot.yaxis.axis_label = 'RF Amplitude (µT)'
bkp.show(plot)

In [ ]:
NFFT = 2 ** 17
freq = np.fft.fftshift(np.fft.fftfreq(NFFT, dt))
# signal = np.append(rf_design.extend(rf_pulse, label_duration / num_reps, dt),
#                    rf_design.extend(rf_pulse * -1, label_duration / num_reps, dt)).real
amp = np.log10(np.abs(np.fft.fftshift(np.fft.fft(rf_sig.real, n=NFFT))) / 1e-6) * 20
# plot RF with bokeh
plot = bkp.figure(width=800, height=400, title='RF pulse')
plot.line(freq, amp, line_width=2)
plot.xaxis.axis_label = 'Frequency (Hz)'
plot.yaxis.axis_label = 'RF Amplitude (µT)'
bkp.show(plot)

In [ ]:
# plot RF and gradients with bokeh
rf_plot = bkp.figure(width=800, height=400, title='RF pulses')
rf_plot.line(time, rf_sig.real * 1e6, line_width=2, alpha=0.5)
rf_plot.line(time, rf_sig.imag * 1e6, line_width=2, color='orange', alpha=0.5)
rf_plot.xaxis.axis_label = 'Time (s)'
rf_plot.yaxis.axis_label = 'RF Amplitude (µT)'

grad_plot = bkp.figure(width=800, height=400, title='Gradient pulses', x_range=rf_plot.x_range)
grad_plot.line(time, G[..., 0, 0, 0, 0], line_width=2, alpha=0.5, color='green')
grad_plot.yaxis.axis_label = 'Gradient Amplitude (T/m)'

bkp.show(bkp.gridplot([[rf_plot], [grad_plot]]))

In [ ]:
mags = bloch.sim(B, T1, T2, duration, dt)

In [ ]:
lab_eff = flow.integrate_across_vessel(radial, bloch.labelling_efficiency(bloch.inverted_magnetization(mags, time, T1, flow_position)), axis=2)[..., 0, 0]

In [ ]:
# time, diastolic flow velocity, systolic flow velocity,
# radial across vessel, off-resonance, B1, space
mags.shape

## Results Visualization

In [ ]:
# bokeh plot for trajectory
p = bkp.figure(width=800, height=300, title='Blood flow', x_range=(flow_time[0], flow_time[-1]))
p.line(flow_time, flow_position[:, 3, 4, 0], line_width=2, legend_label='Position')
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Position (m)'
p.line([flow_time[0], flow_time[-1]], [0] * 2, line_color='red', line_width=2,
       line_dash='dashed', legend_label='Labeling plane')
p.legend.click_policy = 'hide'
p.legend.location = 'top_left'
bkp.output_file('tmp.html')
# bkp.save(p)
bkp.show(p)


In [ ]:
# plot magnetization with bokeh
plot = bkp.figure(width=800, height=400, title='Magnetization')
plot.line(time, mags[:, 3, 4, 0, 0, 0, 0], line_width=2, legend_label='Mx', alpha=0.5)
plot.line(time, mags[:, 3, 4, 0, 0, 0, 1], line_width=2, legend_label='My', color='orange', alpha=0.5)
plot.line(time, mags[:, 3, 4, 0, 0, 0, 2], line_width=2, legend_label='Mz', color='green')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'Magnetization (ref M0)'
plot.x_range = bkm.DataRange1d(start=0, end=duration)
plot.legend.click_policy = 'hide'

# bkp.output_file('magnetization_time_signal.html')
# bkp.save(plot)
bkp.show(plot)

In [ ]:
mags[-1, 3, 4, 0, -1, -1, -1]

In [ ]:
# plot magnetization flow rates with bokeh
title = 'Longitudinal Magnetization History with Pulsatile Blood Flow Diastolic Velocity'
plot = bkp.figure(width=1000, height=500, title=title)
lines = ['solid', 'dashed']
for index in range(end := mags.shape[1]):
    alpha = 1 - index / end
    plot.line(time, mags[:, index, 4, 0, -1, -1, 2], line_width=2, line_dash=lines[index % 2 == 0],
              legend_label=f'{100 * flow_velocity[:, index, 4, 0].min(axis=0):.3g} cm/s',
              alpha=alpha, color='purple')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'Magnetization (ref M0)'
plot.x_range = bkm.DataRange1d(start=0, end=1) # duration)
plot.legend.location = 'bottom_left'
plot.legend.click_policy = 'hide'
bkp.show(plot)


In [ ]:
# plot magnetization flow rates with bokeh
title = 'Longitudinal Magnetization History with Pulsatile Blood Flow Systolic Velocity'
plot = bkp.figure(width=1000, height=500, title=title)
lines = ['solid', 'dashed']
for index in range(2, end := mags.shape[2]):
    alpha = 1 - index / end
    plot.line(time, mags[:, 3, index, 0, -1, -1, 2], line_width=2, line_dash=lines[index % 2 == 0],
              legend_label=f'{100 * flow_velocity[:, 3, index, 0].max(axis=0):.3g} cm/s',
              alpha=alpha, color='purple')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'Magnetization (ref M0)'
plot.x_range = bkm.DataRange1d(start=0, end=1) # duration)
plot.legend.location = 'bottom_left'
plot.legend.click_policy = 'hide'
bkp.show(plot)

In [ ]:
# plot magnetization flow rates with bokeh
title = 'Longitudinal Magnetization History Average Across Vessel with Pulsatile Blood Flow Velocity'
plot = bkp.figure(width=1000, height=500, title=title)
for index in range(0, end := mags.shape[2], end // 10):
    alpha = 1 - index / end
    plot.line(time, mags[:, 3, index, :, -1, -1, 2].mean(axis=1), line_width=2, legend_label=f'{100 * flow_velocity[:, 3, index, 0].max(axis=0):.3g} cm/s',
              alpha=alpha, color='purple')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'Magnetization (ref M0)'
plot.x_range = bkm.DataRange1d(start=0, end=duration)
plot.legend.click_policy = 'hide'
bkp.show(plot)

In [ ]:
# mean before lab eff calc
# lab_eff = labelling_efficiency(inverted_magnetization(mags[..., 0, :, :].mean(axis=2), time, T1, position.mean(axis=2)[..., 0]))

# just mean for lab eff calc
# lab_eff = labelling_efficiency(inverted_magnetization(mags, time, T1, position))[..., 0, :].mean(axis=1)

In [ ]:
# use bokeh to plot labelling efficiency vs systolic velocity
plot = bkp.figure(width=800, height=400, title='Labeling Efficiency vs Systolic and Diastolic Velocity')
for i, dv in enumerate(diastolic_velocity):
    alpha = i / len(diastolic_velocity) + 0.1
    plot.line(systolic * 100, lab_eff[i], line_width=2, alpha=alpha, legend_label=f'Diastolic: {dv * 100:.2f} cm/s')
plot.xaxis.axis_label = 'Systolic Velocity (cm/s)'
plot.yaxis.axis_label = 'Labeling Efficiency'
plot.legend.location = 'bottom_right'
bkp.show(plot)

In [ ]:
# use bokeh to plot labelling efficiency vs systolic velocity
plot = bkp.figure(width=800, height=400, title='Labeling Efficiency vs Systolic Velocity')
# loop over B1 inhomogeneity
for i, b1 in enumerate(B1_homogeneity):
    plot.line(systolic[:, 0] * 100, lab_eff[:, i], line_width=2, alpha=b1, legend_label=f'B1: {b1:.2f}')
plot.xaxis.axis_label = 'Systolic Velocity (cm/s)'
plot.yaxis.axis_label = 'Labeling Efficiency'
plot.legend.location = 'bottom_right'
bkp.show(plot)

In [ ]:
# # plot magnetization off-resonances with bokeh
# title = 'Longitudinal Magnetization with Off-Resonance Pulse'
# plot = bkp.figure(width=1000, height=500, title=title)
# for offres in range(0, end := mags.shape[2], end // 10):
#     alpha = 1 - offres / end
#     plot.line(time, mags[:, flindex, offres, -1, 2], line_width=2, legend_label=f'{dfz[offres]:g} Hz',
#               alpha=alpha, color='green')
# plot.xaxis.axis_label = 'Time (s)'
# plot.yaxis.axis_label = 'Magnetization (ref M0)'
# plot.x_range = bkm.DataRange1d(start=0, end=duration)
# plot.legend.click_policy = 'hide'
# bkp.show(plot)

In [ ]:
# flipped = np.argmin(mags[..., 10, :, -1, 2], axis=0)
# plot = bkp.figure(width=800, height=400, title='Flipped Magnetization Spectrum')
# plot.line(dfz, np.take_along_axis(mags[..., 10, :, -1, 2], flipped[np.newaxis], axis=0)[0],
#           line_width=2, legend_label='Min Mz')
# plot.line(dfz, time[flipped], line_width=2, color='red', legend_label='Time of Min Mz (s)')
# plot.xaxis.axis_label = 'Off-Resonance Frequency (Hz)'
# plot.yaxis.axis_label = 'Magnetization (ref M0)'
# plot.y_range = bkm.DataRange1d(start=-1, end=1)
# plot.legend.click_policy = 'hide'
# bkp.show(plot)

In [ ]:
flow = asnumpy(100 * systolic) # cm/s
dv = asnumpy(100 * diastolic_velocity)

title = 'Labelling Efficiency'
plot = bkp.figure(width=1000, height=500) #, title=title)
color_mapper = bkm.LinearColorMapper(palette='Viridis256', low=0, high=1)
image = plot.image([asnumpy(lab_eff.T)], y=[dv.min()], x=[flow.min()],
                    dh=[dv.max() - dv.min()],
                    dw=[flow.max() - flow.min()], color_mapper=color_mapper)
plot.xaxis.axis_label = 'Pulsatile Blood Flow Systolic Velocity (cm/s)'
plot.yaxis.axis_label = 'Pulsatile Blood Flow Diastolic Velocity (cm/s)'
plot.x_range = bkm.DataRange1d(start=flow.min(), end=flow.max())
plot.y_range = bkm.DataRange1d(start=dv.min(), end=dv.max())

# add colourbar
color_bar = bkm.ColorBar(color_mapper=color_mapper, location=(0, 0))
color_bar.title = title
plot.add_layout(color_bar, 'right')

bkp.output_file(f'{title}.html')
bkp.save(plot)
# bkp.show(plot)

In [ ]:
# ! open "{Out[144]}"

In [ ]:
# flow = asnumpy(100 * flow_velocity.max(axis=-1)) # cm/s
# b1 = asnumpy(B1_homogeneity)

# title = 'Labelling Efficiency'
# plot = bkp.figure(width=1000, height=500) #, title=title)
# color_mapper = bkm.LinearColorMapper(palette='Viridis256', low=0, high=1)
# image = plot.image([asnumpy(lab_eff.T)], y=[b1.min()], x=[flow.min()],
#                     dh=[b1.max() - b1.min()],
#                     dw=[flow.max() - flow.min()], color_mapper=color_mapper)
# plot.xaxis.axis_label = 'Max Pulsatile Blood Flow Velocity (cm/s)'
# plot.yaxis.axis_label = 'B1 Homogeneity'
# plot.x_range = bkm.DataRange1d(start=flow.min(), end=flow.max())
# plot.y_range = bkm.DataRange1d(start=b1.min(), end=b1.max())

# # add colourbar
# color_bar = bkm.ColorBar(color_mapper=color_mapper, location=(0, 0))
# color_bar.title = title
# plot.add_layout(color_bar, 'right')

# bkp.output_file(f'{title}.html')
# bkp.save(plot)
# # bkp.show(plot)

In [ ]:
# save mags, B_field, time compressed numpy file
# np.savez_compressed('asl_simulation.npz', mags=mags[:, 0], B_field=B[:, 0], time=time)